In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [15]:
df_ratings = pd.read_csv("ml-latest-small/ratings.csv",sep=",")

In [16]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [17]:
len(df_ratings.userId.unique()), len(df_ratings.movieId.unique())

(610, 9724)

In [18]:
df_ratings.userId = df_ratings.userId.astype('category').cat.codes.values
df_ratings.movieId = df_ratings.movieId.astype('category').cat.codes.values

In [19]:
df_ratings.head()

,userId,movieId,rating,timestamp
0,0,0,4.0,964982703
1,0,2,4.0,964981247
2,0,5,4.0,964982224
3,0,43,5.0,964983815
4,0,46,5.0,964982931


In [20]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_ratings, test_size=0.2)